In [2]:
import matplotlib.pyplot as plt
import rasterio

img = rasterio.open("ocs-ia-tutoriel/sample/Sauvagnon_64/IMG_087182.jpg").read()
plt.imshow(img.transpose((1, 2, 0)))
plt.axis('off')
plt.show()


ModuleNotFoundError: No module named 'matplotlib'